In [3]:
# coding: utf-8

import random
import pickle

random.seed(1234)

with open('../raw_data/remap.pkl', 'rb') as f:
  reviews_df = pickle.load(f)
  cate_list = pickle.load(f)
  user_count, item_count, cate_count, example_count = pickle.load(f)

In [4]:
reviews_df

,reviewerID,asin,unixReviewTime
0,176008,0,1370131200
1,173739,0,1290643200
2,134504,0,1283990400
3,24476,0,1290556800
4,57419,0,1317254400
...,...,...,...
1689183,107787,63000,1405555200
1689184,22987,63000,1405382400
1689185,58729,63000,1405555200
1689186,19901,63000,1405641600


In [11]:
for reviewerID, hist in reviews_df.groupby('reviewerID'):
  print(hist)
#   user历史上的产品是正样本
  pos_list = hist['asin'].tolist()
  def gen_neg():
    neg = pos_list[0]
    while neg in pos_list:
      print('----')
      neg = random.randint(0, item_count-1)
    return neg

  neg_list = [random.randint(0, item_count-1) for i in range(len(pos_list))]
  print(neg_list)
  break

         reviewerID   asin  unixReviewTime
321546            0  13179      1400457600
450446            0  17993      1400457600
738088            0  28326      1400457600
766200            0  29247      1400457600
1678142           0  62275      1400457600
[30832, 19089, 58306, 27677, 28901]


In [9]:
for reviewerID, hist in reviews_df.groupby('reviewerID'):
  print(hist)
#   user历史上的产品是正样本
  pos_list = hist['asin'].tolist()

  neg_list = [random.randint(0, item_count-1) for i in range(len(pos_list))]
  print(neg_list)
  break

         reviewerID   asin  unixReviewTime
321546            0  13179      1400457600
450446            0  17993      1400457600
738088            0  28326      1400457600
766200            0  29247      1400457600
1678142           0  62275      1400457600
[61908, 20382, 112, 20685, 51376]


In [16]:
train_set = []
test_set = []

# 都是按照人、时间升序排列的
for reviewerID, hist in reviews_df.groupby('reviewerID'):
#   print(hist)
#   user历史上的产品是正样本
  pos_list = hist['asin'].tolist()
  neg_list = [random.randint(0, item_count-1) for i in range(len(pos_list))]

#   之前的是过去的，中间某有个是candidate item，过去的都是行为序列
  for i in range(1, len(pos_list)):
    hist_asin = pos_list[:i]
    if i != len(pos_list) - 1:
      train_set.append((reviewerID, hist_asin, pos_list[i], 1))
      train_set.append((reviewerID, hist_asin, neg_list[i], 0))
    else:
#       每个人一条样本做测试，格式跟train_set的还不一样。形如 (1, [41862, 46010, 54171, 56540, 58134], (62555, 2288))
#       虽然与训练的数据很像
      label = (pos_list[i], neg_list[i])
      test_set.append((reviewerID, hist_asin, label))

In [15]:
test_set[:10]

[(0, [13179, 17993, 28326, 29247], (62275, 5940)),
 (1, [41862, 46010, 54171, 56540, 58134], (62555, 2288)),
 (2, [42298, 42390, 46782, 47355], (50682, 50232)),
 (3, [21989, 25578, 58444, 60072], (62274, 51693)),
 (4, [3112, 30275, 40094, 48963, 54245, 58671], (62022, 30379)),
 (5, [24157, 28174, 30462, 30573, 50847, 55698, 58911, 59196], (60625, 31899)),
 (6, [20912, 25945, 26644, 29206, 42564, 43676], (47798, 30533)),
 (7, [10064, 13629, 31874, 32991, 38943], (56441, 37945)),
 (8, [12617, 13540, 36841, 47477, 54708], (60479, 42246)),
 (9, [12023, 15970, 22602, 23519, 31488], (53243, 16328))]

In [17]:
random.shuffle(train_set)
random.shuffle(test_set)

assert len(test_set) == user_count
# assert(len(test_set) + len(train_set) // 2 == reviews_df.shape[0])

with open('dataset.pkl', 'wb') as f:
  pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
  pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)   
  pickle.dump(cate_list, f, pickle.HIGHEST_PROTOCOL)
  pickle.dump((user_count, item_count, cate_count), f, pickle.HIGHEST_PROTOCOL)